In [47]:
# Basic Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sqlite3
from scipy import stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

# Magic lines
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [48]:
df = pd.read_csv('../../data/raw/Airbnb_Open_Data.csv')
df.head()

C:\Users\Lucas\AppData\Local\Temp\ipykernel_39988\4054701482.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/raw/Airbnb_Open_Data.csv')


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [49]:
df.shape

(102599, 26)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [51]:
df.describe()

,id,host id,lat,long,Construction year,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365
count,1.025990e+05,1.025990e+05,102591.000000,102591.000000,102385.000000,102190.000000,102416.000000,86720.000000,102273.000000,102280.000000,102151.000000
mean,2.914623e+07,4.925411e+10,40.728094,-73.949644,2012.487464,8.135845,27.483743,1.374022,3.279106,7.936605,141.133254
std,1.625751e+07,2.853900e+10,0.055857,0.049521,5.765556,30.553781,49.508954,1.746621,1.284657,32.218780,135.435024
min,1.001254e+06,1.236005e+08,40.499790,-74.249840,2003.000000,-1223.000000,0.000000,0.010000,1.000000,1.000000,-10.000000
25%,1.508581e+07,2.458333e+10,40.688740,-73.982580,2007.000000,2.000000,1.000000,0.220000,2.000000,1.000000,3.000000
50%,2.913660e+07,4.911774e+10,40.722290,-73.954440,2012.000000,3.000000,7.000000,0.740000,3.000000,1.000000,96.000000
75%,4.320120e+07,7.399650e+10,40.762760,-73.932350,2017.000000,5.000000,30.000000,2.000000,4.000000,2.000000,269.000000
max,5.736742e+07,9.876313e+10,40.916970,-73.705220,2022.000000,5645.000000,1024.000000,90.000000,5.000000,332.000000,3677.000000


# Preprocessing

In [52]:
df.duplicated().sum()

541

In [53]:
df.duplicated().sum()/len(df)

0.0052729558767629315

In [54]:
df.isnull().sum().sort_values(ascending=False)

license                           102597
house_rules                        52131
last review                        15893
reviews per month                  15879
country                              532
availability 365                     448
minimum nights                       409
host name                            406
review rate number                   326
calculated host listings count       319
host_identity_verified               289
service fee                          273
NAME                                 250
price                                247
Construction year                    214
number of reviews                    183
country code                         131
instant_bookable                     105
cancellation_policy                   76
neighbourhood group                   29
neighbourhood                         16
long                                   8
lat                                    8
id                                     0
host id         

## Drop duplicates

In [55]:
df = df.drop_duplicates(ignore_index=False)

# Traitement des NaN

Pour le traitement des NaN nous allons raisonner par colonnes

Les colonnes license et house_rules ne sont pas nécessaires à nos prochaines analyse

In [56]:
df = df.drop(columns=['license', 'house_rules'])

On supprime les lignes qui contiennent des NaN dans les colonnes price et neighbourhood group car elles ne sont pas utilisables

In [57]:
df = df.dropna(subset=['price', 'neighbourhood group'])

Fillna pour les reviews per month

In [58]:
df['reviews per month'] = df['reviews per month'].fillna(0)

## Traitement des formats

Nous avons les colonnes price et service fees qui sont en object. Nous devons les passer en float.

In [59]:
df['price'].head(20)

0       $966 
1       $142 
2       $620 
3       $368 
4       $204 
5       $577 
6        $71 
7     $1,060 
8     $1,018 
9       $291 
10      $319 
11      $606 
12      $714 
13      $580 
14      $149 
15      $578 
16      $778 
17      $656 
18      $460 
19    $1,095 
Name: price, dtype: object

In [60]:
df['service fee'].head(20)

0     $193 
1      $28 
2     $124 
3      $74 
4      $41 
5     $115 
6      $14 
7     $212 
8     $204 
9      $58 
10     $64 
11    $121 
12    $143 
13    $116 
14     $30 
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
Name: service fee, dtype: object

In [61]:
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['service fee'] = df['service fee'].replace('[\$,]', '', regex=True).astype(float)


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101783 entries, 0 to 102057
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              101783 non-null  int64  
 1   NAME                            101538 non-null  object 
 2   host id                         101783 non-null  int64  
 3   host_identity_verified          101506 non-null  object 
 4   host name                       101385 non-null  object 
 5   neighbourhood group             101783 non-null  object 
 6   neighbourhood                   101768 non-null  object 
 7   lat                             101775 non-null  float64
 8   long                            101775 non-null  float64
 9   country                         101257 non-null  object 
 10  country code                    101658 non-null  object 
 11  instant_bookable                101684 non-null  object 
 12  cancellation_policy  

In [ ]:
df['price'].head(20)

0      966.0
1      142.0
2      620.0
3      368.0
4      204.0
5      577.0
6       71.0
7     1060.0
8     1018.0
9      291.0
10     319.0
11     606.0
12     714.0
13     580.0
14     149.0
15     578.0
16     778.0
17     656.0
18     460.0
19    1095.0
Name: price, dtype: float64

: 